In [65]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmbb-movies-5000/tmdb_movies3.csv


In [68]:


import requests
import pandas as pd

API_KEY = '2f1149e40c1cf8712d8658d30cfd89c0'
BASE_URL = 'https://api.themoviedb.org/3'
MOVIES_PER_PAGE = 20
TOTAL_MOVIES = 5000
TOTAL_PAGES = TOTAL_MOVIES // MOVIES_PER_PAGE

def fetch_movies(api_key, page=1):
    url = f"{BASE_URL}/movie/popular?api_key={api_key}&language=en-US&page={page}"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def fetch_movie_details(api_key, movie_id):
    url = f"{BASE_URL}/movie/{movie_id}?api_key={api_key}&language=en-US"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def extract_movie_data(movies):
    movie_list = []
    for movie in movies:
        movie_data = fetch_movie_details(API_KEY, movie['id'])
        movie_dict = {
            'id': movie_data.get('id'),
            'title': movie_data.get('title'),
            'release_date': movie_data.get('release_date'),
            'vote_average': movie_data.get('vote_average'),
            'vote_count': movie_data.get('vote_count'),
            'popularity': movie_data.get('popularity'),
            'original_language': movie_data.get('original_language'),
            'genre_ids': movie_data.get('genres', []),
            'budget': movie_data.get('budget'),
            'revenue': movie_data.get('revenue'),
            'runtime': movie_data.get('runtime'),
            'production_companies': [company['name'] for company in movie_data.get('production_companies', [])],
            'overview': movie_data.get('overview')
        }
        movie_list.append(movie_dict)
    return movie_list

def main():
    all_movies = []
    for page in range(1, TOTAL_PAGES + 1):
        movies_data = fetch_movies(API_KEY, page)
        movies = movies_data.get('results', [])
        all_movies.extend(extract_movie_data(movies))
        print(f"Fetched page {page}")
        
        if len(all_movies) >= TOTAL_MOVIES:
            break

    # Trim the list to the exact number of desired movies if overshot
    all_movies = all_movies[:TOTAL_MOVIES]

    # Create a DataFrame
    df = pd.DataFrame(all_movies)
    # Save to CSV
    df.to_csv('tmdb_movies1.csv', index=False)
    print("Data saved to tmdb_movies2.csv")

if __name__ == "__main__":
    main()

Fetched page 1
Fetched page 2
Fetched page 3
Fetched page 4
Fetched page 5
Fetched page 6
Fetched page 7
Fetched page 8
Fetched page 9
Fetched page 10
Fetched page 11
Fetched page 12
Fetched page 13
Fetched page 14
Fetched page 15
Fetched page 16
Fetched page 17
Fetched page 18
Fetched page 19
Fetched page 20
Fetched page 21
Fetched page 22
Fetched page 23
Fetched page 24
Fetched page 25
Fetched page 26
Fetched page 27
Fetched page 28
Fetched page 29
Fetched page 30
Fetched page 31
Fetched page 32
Fetched page 33
Fetched page 34
Fetched page 35
Fetched page 36
Fetched page 37
Fetched page 38
Fetched page 39
Fetched page 40
Fetched page 41
Fetched page 42
Fetched page 43
Fetched page 44
Fetched page 45
Fetched page 46
Fetched page 47
Fetched page 48
Fetched page 49
Fetched page 50
Fetched page 51
Fetched page 52
Fetched page 53
Fetched page 54
Fetched page 55
Fetched page 56
Fetched page 57
Fetched page 58
Fetched page 59
Fetched page 60
Fetched page 61
Fetched page 62
Fetched page 63
F

In [3]:


import pandas as pd
import numpy as np
import requests
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# Load the CSV file
file_path = '/kaggle/input/tmbb-movies-5000/tmdb_movies3.csv'
movies_df = pd.read_csv(file_path)

# Add a random user ID to simulate user ratings (for demonstration purposes)
movies_df['userid'] = np.random.randint(1, 6, movies_df.shape[0])

# Keep relevant columns
movies_df = movies_df[['id', 'userid', 'vote_count', 'runtime', 'title']]

# Display basic statistics
print(movies_df.describe())

# Calculate user frequency (number of votes per user)
user_freq = movies_df[['userid', 'id']].groupby('userid').count().reset_index()
user_freq.columns = ['userid', 'vote_count']
print(user_freq.head())

# Calculate mean vote counts for each movie
mean_rating = movies_df.groupby('id')[['vote_count']].mean()

# Identify the lowest and highest rated movies
lowest_rated = mean_rating['vote_count'].idxmin()
highest_rated = mean_rating['vote_count'].idxmax()
print(f"Lowest rated movie ID: {lowest_rated}")
print(f"Highest rated movie ID: {highest_rated}")

# Show details of the lowest and highest rated movies
print(movies_df.loc[movies_df['id'] == lowest_rated])
print(movies_df.loc[movies_df['id'] == highest_rated])

# Calculate movie statistics for Bayesian average
movie_stats = movies_df.groupby('id')[['vote_count']].agg(['count', 'mean'])
movie_stats.columns = movie_stats.columns.droplevel()

# Function to create a user-movie matrix
def create_matrix(df):
    N = len(df['userid'].unique())
    M = len(df['id'].unique())

    # Map IDs to indices
    user_mapper = {userid: idx for idx, userid in enumerate(np.unique(df["userid"]))}
    movie_mapper = {movieid: idx for idx, movieid in enumerate(np.unique(df["id"]))}

    # Reverse mapping of indices to IDs
    user_inv_mapper = {idx: userid for userid, idx in user_mapper.items()}
    movie_inv_mapper = {idx: movieid for movieid, idx in movie_mapper.items()}

    user_index = [user_mapper[userid] for userid in df['userid']]
    movie_index = [movie_mapper[movieid] for movieid in df['id']]

    # Create sparse matrix
    X = csr_matrix((df["vote_count"], (movie_index, user_index)), shape=(M, N))

    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

# Create user-movie matrix
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_matrix(movies_df)

# Function to find similar movies using K-Nearest Neighbors
def find_similar_movies(movie_id, X, k=10, metric='cosine', show_distance=False):
    neighbour_ids = []

    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    k += 1  # Increase k by 1 to exclude the movie itself

    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)

    movie_vec = movie_vec.reshape(1, -1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)

    for i in range(1, k):
        n = neighbour[1].item(i) if show_distance else neighbour[0].item(i)
        neighbour_ids.append(movie_inv_mapper[n])

    return neighbour_ids

# Dictionary to map movie IDs to titles
movie_titles = dict(zip(movies_df['id'], movies_df['title']))

# Example usage of the recommendation system
movie_id = 629176
similar_ids = find_similar_movies(movie_id, X, k=10)
movie_title = movie_titles[movie_id]

print(f"Since you watched {movie_title}")
for i in similar_ids:
    print(movie_titles[i])




                 id       userid   vote_count      runtime
count  5.000000e+03  5000.000000   5000.00000  5000.000000
mean   3.158422e+05     2.973000   2799.12940   105.692400
std    3.691515e+05     1.415016   3990.58005    26.525612
min    5.000000e+00     1.000000      0.00000     0.000000
25%    1.019275e+04     2.000000    335.75000    93.000000
50%    1.095055e+05     3.000000   1362.00000   104.000000
75%    5.551300e+05     4.000000   3553.00000   119.000000
max    1.294849e+06     5.000000  35823.00000   260.000000
   userid  vote_count
0       1        1016
1       2        1044
2       3         981
3       4         977
4       5         982
Lowest rated movie ID: 83533
Highest rated movie ID: 27205
         id  userid  vote_count  runtime     title
3218  83533       2           0        0  Avatar 3
3225  83533       1           0        0  Avatar 3
       id  userid  vote_count  runtime      title
50  27205       1       35823      148  Inception
Since you watched Samarit